<h1>Decision Trees and Machine Learning</h1>

<li>Decision trees are tree structures containing rules</li>
<li>The leaf nodes of the tree are the "learned" categories (or threshold values)</li>
<li>A path from the root to a leaf node represents a rule (or a decision path)</li>
<li>Leaf nodes represent predictions (either a category or the mean (expected) value of a decision variable</li>
<li>Each case filters through the tree from the root to a leaf node to get a prediction</li>

<span style="color:green;font-size:xx-large">Why decision trees?</span>
<li>Easy to understand </li>
<li>Rule finding process is transparent</li>
<li>Can handle "mixed" categorical(male/female) and numerical (age, number of siblings) data</li>
<li>Can handle missing data </li>
<li>Can be used to generate partial "good" solutions</li>
<li>Can find non-linear patterns in the data</li>

<span style="color:green;font-size:xx-large">Decision trees and non-linear patterns</span>
<p></p>
<img src="linear_model.png">
<img src="linear_classifier_accuracy.png">
<img src="non_linear_classifier.png">
<img src="decision_tree.png">

<span style="color:green;font-size:xx-large">Why not decision trees?</span>
<li>Finding an optimal tree is a hard problem</li>
<li>Overfitting is a problem</li>

<h1>Example: Predicting wine quality</h1>
<li><span style="color:blue">Input features</span>: Chemical properties of wines</li>
<li><span style="color:blue">Wine quality</span>: A number between 0 and 10</li>

<span style="color:green;font-size:x-large">Import the data</span>

In [1]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
import pandas as pd
from pandas import DataFrame
w_df = pd.read_csv(url,header=0,sep=';')
w_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


<span style="color:green;font-size:x-large">Examine the dependent variable</span>
<li>Higher dv values indicate a better quality wine</li>
<li>Lower dv values indicate a poorer quality wine</li>
<li>We'll assume that the values are continuous</li>
<li>And use the various features to predict wine quality</li>

In [2]:
w_df['quality'].unique()

array([5, 6, 7, 4, 8, 3])

Because our quality, the variable we are trying to classify by, is an ordered set, we cannot use traditional classification trees, we need to use regression trees.

<span style="color:green;font-size:x-large">Build train and test samples</span>

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(w_df, test_size = 0.3)
x_train_wine = train.iloc[0:,0:11]
y_train_wine = train[['quality']]
x_test_wine = test.iloc[0:,0:11]
y_test_wine = test[['quality']]



<span style="color:green;font-size:xx-large">Choose a model</span>
<p></p>
<ul>
<li><b>Classification trees</b>: Uses rules to classify cases into two or more categories (classify handwritten digits)</li>
<ul>
<li>Classification trees recursively split the data on a feature value</li>
<li>Each split minimizes the entropy (also known as the impurity)</li>
<li>Entropy is commonly measured using the GINI cost function (a measure of the probability of misclassification or 'purity')</li>
    <li>Classifiers are used when the target variable is a set of unordered categories (handwritten digit recognition)</li>
</ul>
<li><b>Regression trees</b>: Uses rules to group data into target variable ranges (Wine Quality)</li>
<ul>
<li>Also split the data on feature values</li> 
    <li>Minimize cost (impurity). Usually the mean squared error</li>
    <li>Regression trees are used when the target variable is continuous and ordered (wine quality from 0 to 10)</li>
    </ul>
</ul>

<span style="color:green;font-size:x-large">Regression trees</span>
<li>Regression trees use a greedy least squares algorithm to construct the tree</li>
<li>The most common algorithm, roughly described below, is CART (Classification and Regression Trees) and this is what sklearn uses</li>
<li>For each variable and each possible value the variable can take, calculate the mean square error generated by the split</li>
<ul>
<li>Since many variables are continuous, the set of possible values is pseudo continuous (the algorithm uses a finite subset of possible values, usually the set of actual values in the data)</li>
</ul>
<li>Calculate the Mean Square Error of each of the two halves for each split, and pick the split with the lowest MSE</li>
<ul>
    <li>The MSE is the square of the difference between each <i>actual</i> value in a half and the mean of all actual values in that half</li>
    <li>The comined MSE is the sum of the two MSEs (each half of the split)</li>
    <li> $ mse_{split} = \Sigma_{i\in S_{left}} (y_{i} - \bar{y})^{2} + \Sigma_{i\in S_{right}} (y_{i} - \bar{y})^{2} $ </li>
    <li>Each (predictor,value) combination generates a split and the combination with the lowest MSE is chosen, generating two nodes, a left node and a right node</li>
    <li>The process is then repeated at the left and right nodes until some stopping criterion is met</li>


<span style="color:green;font-size:x-large">Classification trees</span>
<li>Classification trees use a greedy entropy minimization algorithm to construct the tree</li>
<li><b>Entropy</b>: is a measure of uncertainty in the data<p></li>
<ul>
    <li>If there are c classes and the probability of drawing an item from class i is given by $ p_{i} $, then the entropy of the system is:
        <ul>
            <li> $E = \sum_{i=1}^{c} -p_{i}\times log_{2} p_{i} $ </li>
        </ul>
        <li>what is the uncertainty when you draw a marble from a box with 50 blue and 50 red marbles?</li>
    <ul>
        <li> $ p_{b} = 0.5 $</li>
        <li> $ p_{r} = 0.5 $</li>
        <li> $ E = (-0.5 \times log_{2} 0.5) + (-0.5 \times log_{2} 0.5) $</li>
        <li> or, $ -0.5 \times -1 + -0.5 * -1 = 1.0 $</li>
    </ul>
    <li>what is the uncertainty when you draw a marble from a box with 60 blue and 40 red marbles?</li>
    <ul>
        <li> $ p_{b} = 0.6 $</li>
        <li> $ p_{r} = 0.4 $</li>
        <li> $ E = -0.6 \times log_{2} 0.6 + -0.4 \times log_{2} 0.4 $</li>
        <li> or, $ (-0.6 \times -0.737) + (-0.4 \times -1.322) = 0.971$</li>
    </ul>
    <li>what is the uncertainty in color when you draw a marble from a box of 100 blue marbles?</li>
    <ul>
        <li> $ p_{b} = 1.0 $</li>
        <li> $ E = 1.0 \times log_{2} 1.0 $</li>
        <li> or, 0.0 </li>
    </ul>
    

</ul>

<span style="color:green;font-size:xx-large">Information gain vs Gini in classification trees</span>
<p></p>
<li>Classification trees use either information gain or Gini index in selecting splits</li>
<li><b>Information gain</b> refers to the reduction in entropy resulting from a split. Classification trees use information gain as the decision factor when choosing a split</li>
<ul> <li>$ e_{parent} - (w_{left} \times e_{left} + w_{right} \times e_{right}) $ where e is entropy and w is the weight given to a subnode</li>
</ul>
<li>The split with the maximum information gain is selected</li>
<li><b>Gini index</b> aka <b>gini impurity</b> is an entropy measure that uses the probability that a selected data element is classified incorrectly</li>
<ul>
    <li> $ gini = \sum_{i=1}^{c} p_{i}^{2} $ where c is the number of classes</li>
    <li> $ giniimpurity = 1 - gini $</li>

    

<span style="color:green;font-size:x-large">Understanding Gini splits</span>
<li>Example: predicting the grade of a student in a class</li>
<li>One independet feature: attendance, a binary feature which takes two values A: Attend regularly or S: Skip often</li>
<li>The following diagram represents the split for which we want to calculate impurity</li>
<img src="gini.png">


<li> $ gini_{left} = (0.57)^{2} + (0.43)^{2} = 0.51 $ </li>
<li> $ giniimpurity_{left} = 0.49 $ </li>
<li> $ gini_{right} = (0.33)^{2} + (0.67)^{2} = 0.56 $ </li>
<li> $ giniimpurity_{right} = 0.44 $</li>
<li> $ w_{left} = \frac{14}{20}$ </li>
<li> $ w_{right} = \frac{6}{20}$ </li>
<li> $ gi_{split} = \frac{14}{20} \times 0.49 + \frac{6}{20} \times 0.44 = 0.475 $ </li>
<li> CART tests all combinations of features and feature values and picks the split with the lowest impurity</li>


<span style="color:green;font-size:x-large">Tree depth: Stopping and Pruning Rules</span>
<li>In the degenerate case, a decision tree algorithm can build a tree with exactly one training case in each leaf node</li>
<li>This would be pointlessly overfitted!</li>
<p></p>
<span style="color:green;font-size:x-large">controlling for overfitting</span>

<li>Set a minimum count of observations in each leaf node. If the number of observations falls below the minimum, don't split the node any further</li>
<li>Set a maximum tree <b>depth</b>. Once a path reaches a certain length, stop splitting that path</li>
<li>Minimize <b>complexity cost</b>. Complexity cost in a decision tree is a function of the overall misclassification rate of the tree (we want the overall misclasification rate to be low) and the number of leaf nodes (we don't want too many categories because of the overfitting danger)</li>
<li>Various other parameters (cf. <a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html">https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html</a>)</li>

<span style="color:green;font-size:x-large">The wine rating problem</span>
<li>Since our dv is ordered and continuous, we'll choose a regression tree</li>

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
wine_rgr = tree.DecisionTreeRegressor(max_depth=3,min_samples_leaf=20,min_samples_split=50)
wine_rgr.fit(x_train_wine,y_train_wine)

<span style="color:green;font-size:x-large">Evaluating a regressor</span>
<li>Since a regressor is predicting continuous values of the dv, we can evaluate it like a linear regression model</li>
<li><span style="color:blue">R-Square</span> tells us how much of the variance in the data is explained by our model (how well the model fits the data)</li>
<li><span style="color:blue">mean square error</span> gives us an estimate of how far, on the average, are our predictions from actuals (this is better as a model comparison tool)</li>

In [ ]:
#Get the R-Square for the predicted vs actuals on the text sample
print("Training R-Square",wine_rgr.score(x_train_wine,y_train_wine))
print("Testing R-Square",wine_rgr.score(x_test_wine,y_test_wine))
#print("Training mean sq error",wine_rgr.score(x_train_wine,y_train_wine))
#print("Testing mean sq error",wine_rgr.score(x_test_wine,y_test_wine))

<span style="color:green;font-size:x-large">Visualizing the tree</span>
<li>sklearn has a handy function <a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html">plot_tree</a> for visualizing a tree</li>

In [ ]:
import matplotlib.pyplot as plt

#associate column names with features
feature_names = w_df.columns[:-1]

#set up a figure (mainly for the size)
fig, ax = plt.subplots(figsize=(24, 28))

#plot the tree
tree.plot_tree(wine_rgr,feature_names=feature_names, max_depth=4, fontsize=12)

plt.show()



<h1>Classification trees</h1>



<span style="color:green;font-size:xx-large">In-class problem: Classifying wine into good or bad wine</span>
<li>assume any wine with quality less than 5.5 is a bad wine</li>
<li>and any wine with quality greater than or equal to 5.5 is a good wine</li>
<li>build a classifier that classifies the wine data into good and bad wines</li>
<li>use the training data to build the classifier and report the accuracy <span style="color:red">score</span> on the testing data</li>

In [ ]:
#Build train and test data
x_train_wine = train.iloc[0:,0:11]
y_train_wine = train[['quality']]
x_test_wine = test.iloc[0:,0:11]
y_test_wine = test[['quality']]

#Convert y values into categorical data
y_train_wine_cat = y_train_wine >= 5.5
y_test_wine_cat = y_test_wine >= 5.5

#Set up and fit a model 
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
wine_clf = DecisionTreeClassifier(max_depth=4)
#fit the data to the classifier
wine_clf.fit(x_train_wine,y_train_wine_cat)

#Report the accuracy score
training_accuracy = wine_clf.score(x_train_wine,y_train_wine_cat)
testing_accuracy = wine_clf.score(x_test_wine,y_test_wine_cat)
print(training_accuracy)
print(testing_accuracy)

#Render the decision tree
import matplotlib.pyplot as plt
feature_names = w_df.columns[:-1]
fig, ax = plt.subplots(figsize=(24, 28))
tree.plot_tree(wine_clf,feature_names=feature_names, max_depth=4, fontsize=12)

plt.show()
